In [ ]:
#   Copyright (c) 2020, Xilinx, Inc.
#   All rights reserved.
#
#   Redistribution and use in source and binary forms, with or without
#   modification, are permitted provided that the following conditions are met:
#
#   1.  Redistributions of source code must retain the above copyright notice,
#       this list of conditions and the following disclaimer.
#
#   2.  Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#   3.  Neither the name of the copyright holder nor the names of its
#       contributors may be used to endorse or promote products derived from
#       this software without specific prior written permission.
#
#   THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
#   AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO,
#   THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
#   PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR
#   CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
#   EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
#   PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS;
#   OR BUSINESS INTERRUPTION). HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
#   WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR
#   OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF
#   ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

VIO Example
===========

Description
-----------
This demo shows how to do three things:
1. Connect to a Versal target device via ChipScope Server (cs_server)
   and Hardware Server (hw_server)
2. Program a Versal target device using a design PDI file
3. Communicate with a VIO debug core in hardware

Requirements
------------
The following is required to run this demo:
1. Local or remote access to a Versal device
2. cs_server and hw_server applications
3. Python 3.7 environment
4. Pip-installed chipscopy python package

---

## Step 1 - Set up environment

In [1]:
import os
from chipscopy import get_examples_dir_or_die

# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")
BOARD = os.getenv("HW_SERVER_BOARD", "vck190/production/2.0")

EXAMPLES_DIR = get_examples_dir_or_die()
PDI_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.pdi"
assert os.path.isfile(PDI_FILE)
LTX_FILE = f"{EXAMPLES_DIR}/designs/{BOARD}/ks_demo/ks_demo_wrapper.ltx"
assert os.path.isfile(LTX_FILE)

print(f"HW_URL={HW_URL}")
print(f"CS_URL={CS_URL}")
print(f"PDI={PDI_FILE}")
print(f"LTX={LTX_FILE}")

HW_URL=TCP:gomphus2:3121
CS_URL=TCP:xsjmdarnall40x:3042
PDI=C:\Users\mdarnall\dev\git\chipscopy_dev\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi
LTX=C:\Users\mdarnall\dev\git\chipscopy_dev\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.ltx


## Step 2 - Create a session and connect to the server(s)
Here we create a new session and print out version information for diagnostic purposes.
The session is a container that keeps track of devices and debug cores.

In [2]:
import chipscopy
from chipscopy import create_session, report_versions

print(f"Using chipscopy api version: {chipscopy.__version__}")
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

Using chipscopy api version: 2020.3.1601921484


## Step 3 - Get our device from the session

In [3]:

# Use the first available device and setup its debug cores
if len(session.devices) == 0:
    raise ValueError("No devices detected")
versal_device = session.devices[0]

## Step 4 - Program the device with our example PDI

In [4]:
print(f"Programming {PDI_FILE}...")
versal_device.program(PDI_FILE)

Programming C:\Users\mdarnall\dev\git\chipscopy_dev\chipscopy\chipscopy\examples/designs/vck190/production/2.0/ks_demo/ks_demo_wrapper.pdi...



Output()

## Step 5 - Detect Debug Cores

In [5]:

print(f"Discovering debug cores...")
versal_device.discover_and_setup_cores(ltx_file=LTX_FILE)

vio_count = len(versal_device.vio_cores)
print(f"\nFound {vio_count} VIO cores in design")
assert vio_count == 2

Discovering debug cores...

Found 2 VIO cores in design


### Find the VIO cores
This shows some different ways to find the VIO core - by name, by uuid, or by index
and how to print port names and probe names.

In [7]:
vio_cores = versal_device.vio_cores
for index, vio_core in enumerate(vio_cores):
    print(f"\nVIO Core Index {index}")
    print("NAME       :", vio_core.name)
    print("UUID       :", vio_core.uuid)
    print("PORT_NAMES :", vio_core.port_names)
    print("PROBE_NAMES:", vio_core.probe_names)

# Get VIO by uuid:
vio_by_uuid = versal_device.vio_cores.filter_by(uuid="499D80541CE65035B25919B9E0CD7838")[0]

my_vio = vio_by_uuid


VIO Core Index 0
NAME       : VIO_4
UUID       : F77F6EBA28EE5440BA5736D4147F4955
PORT_NAMES : ['probe_in0', 'probe_out0', 'probe_out1', 'probe_out2', 'probe_out3', 'probe_out4']
PROBE_NAMES: ['ks_demo_i/fast_counter_0_Q', 'ks_demo_i/fast_counter_0_CE', 'ks_demo_i/fast_counter_0_SCLR', 'ks_demo_i/fast_counter_0_UP', 'ks_demo_i/fast_counter_0_LOAD', 'ks_demo_i/fast_counter_0_L']

VIO Core Index 1
NAME       : VIO_5
UUID       : 499D80541CE65035B25919B9E0CD7838
PORT_NAMES : ['probe_in0', 'probe_out0', 'probe_out1', 'probe_out2', 'probe_out3', 'probe_out4']
PROBE_NAMES: ['ks_demo_i/slow_counter_0_Q', 'ks_demo_i/slow_counter_0_CE', 'ks_demo_i/slow_counter_0_SCLR', 'ks_demo_i/slow_counter_0_UP', 'ks_demo_i/slow_counter_0_LOAD', 'ks_demo_i/slow_counter_0_L']


## Step 6 - Read and Write Ports
- Write known value to port "probe_out4"
- Verify value was written to port "probe_out4"
- Reset port to default value

In [8]:
my_vio.reset_vio()
print("\nPort Data before write:", my_vio.read_ports(["probe_out4"]))
new_port_values = {"probe_out4": 12345678}
my_vio.write_ports(new_port_values)
print(f"Wrote {12345678} to probe_out4")
print("Port Data after write:", my_vio.read_ports(["probe_out4"]))


Port Data before write: {'probe_out4': {'value': 0}}
Wrote 12345678 to probe_out4
Port Data after write: {'probe_out4': {'value': 12345678}}


## Step 7 - Read and Write Probes
- Port probe_out4 is the same as probe ks_demo_i/slow_counter_0_L
- Write known value to port probe_out4
- Read value back from probe ks_demo_i/slow_counter_0_L
- Verify value is the same as written to probe_out4

In [9]:
my_vio.reset_vio()
print("\nPort Data before write:", my_vio.read_ports(["probe_out4"]))
print("Probe Data before write:", my_vio.read_probes(["ks_demo_i/slow_counter_0_L"]))
my_vio.write_ports(new_port_values)
new_port_values = {"probe_out4": 12345678}
my_vio.write_ports(new_port_values)
print(f"Wrote {12345678} to probe_out4")
print("Port Data after write:", my_vio.read_ports(["probe_out4"]))
print("Probe Data after write:", my_vio.read_probes(["ks_demo_i/slow_counter_0_L"]))


Port Data before write: {'probe_out4': {'value': 0}}
Probe Data before write: {'ks_demo_i/slow_counter_0_L': 0}
Wrote 12345678 to probe_out4
Port Data after write: {'probe_out4': {'value': 12345678}}
Probe Data after write: {'ks_demo_i/slow_counter_0_L': 12345678}
